In [1]:
from langgraph.graph import StateGraph , START , END
from langchain_google_genai import  ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

/home/powder-03/langgraph/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    
    sr: float
    bpb: float
    boundary_percent: float
    

In [ ]:
graph = StateGraph(BatsmanState)

graph.add_node('calculate_sr' , calculate_sr)
graph.add_node('calculate_bpb' , calculate_bpb)
graph.add_node('calculate_boundary_percent' , calculate_boundary_percent)
graph.add_node('summary' , summary)